## Connection

In [1]:
import os

from dotenv import load_dotenv
import numpy as np
import pandas as pd
from sqlalchemy import create_engine


# creds from .env file
load_dotenv()

POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_USERNAME = os.getenv('POSTGRES_USERNAME')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_DBNAME = os.getenv('POSTGRES_DBNAME')


# connection
try:
    conn = create_engine(f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    print('connected!')
except:
    print('Can`t establish connection to database')


# small function for selecting data
def select(sql):
    return pd.read_sql_query(sql, conn)

connected!


## UTM / Campaigns

In [2]:
sql = ''' 
select distinct utm_medium
from users;
'''

select(sql)

,utm_medium
0,cpc
1,


In [3]:
sql = ''' 
SELECT 
  utm_medium,
  COUNT(*) AS users_count,
  COUNT(CASE WHEN status = 'customer' THEN id END) AS customers_count
FROM users
GROUP BY 1
'''

select(sql)

,utm_medium,users_count,customers_count
0,cpc,499,134
1,,5583,640


Conversion rate - is basically a percentage of users who currently are customers.

In [4]:
sql = ''' 
WITH channel_stats AS (
  SELECT 
    utm_medium,
    COUNT(*) AS users_count,
    COUNT(CASE WHEN status = 'customer' THEN id END) AS customers_count
  FROM users
  GROUP BY 1
)

SELECT
  utm_medium,
  (1.0 * customers_count / users_count) AS customer_rate
FROM channel_stats
'''

select(sql)

,utm_medium,customer_rate
0,cpc,0.268537
1,,0.114634


In [8]:
sql = ''' 
select count(*)
from users 
where utm_source = 'twitter';

'''

select(sql)

,count
0,421


In [13]:
sql = ''' 
select 
    utm_campaign, 
    count(*)
from users
group by utm_campaign 
order by 2 desc
limit 2;
'''
select(sql)

,utm_campaign,count
0,,5583
1,20180208_us-ca-gb-au_mf_bindle-launch,300


In [ ]:
sql = ''' 
SELECT
  (CASE WHEN utm_campaign IS NULL THEN 'organic' ELSE 'paid' END) AS signup_type,
  *
FROM users
'''
select(sql)

In [23]:
sql = ''' 
select 
	utm_campaign
	, count(*)
from users
where created_at::date between '2018-02-01' and '2018-02-28' 
group by 1

'''
select(sql)

,utm_campaign,count
0,,1812
1,20180201_us_mf_reading,78
2,20180205_ca_mf_online_reading,119
3,20180208_us-ca-gb-au_mf_bindle-launch,300


In [32]:
sql = ''' 
select 
    utm_content
    , utm_source
    , count(id)
from users
where utm_source = 'twitter'

group by 1, 2
order by 2 desc

    
'''
select(sql)

,utm_content,utm_source,count
0,bindle_was_launched,twitter,226
1,discover_best_books,twitter,102
2,new_way_to_read_online,twitter,93


## Purchases

In [34]:
sql = ''' 
SELECT *
FROM purchases
'''
purchases = select(sql)
purchases.head()

,id,user_id,product_id,amount,refunded,created_at
0,1,2306,2,14.99,f,2018-11-25 23:29:23.063394
1,2,2316,2,14.99,f,2018-05-17 21:35:18.482118
2,3,2355,2,14.99,f,2018-10-21 21:09:35.54307
3,4,2374,2,14.99,f,2018-09-23 01:11:33.904443
4,5,2416,2,14.99,f,2018-12-04 18:01:14.317714


In [36]:
sql = ''' 
SELECT 
  SUM(amount) AS net_revenue
FROM purchases
WHERE
  refunded = FALSE

'''
select(sql)

,net_revenue
0,45726.65


In [48]:
sql = ''' 
SELECT *
FROM marketing_spends
limit 3;
'''
select(sql)

,id,spent_at,amount,utm_medium,utm_source,utm_campaign,utm_term,utm_content,clicks
0,1,2018-02-08,76.79,cpc,twitter,20180208_us-ca-gb-au_mf_bindle-launch,,bindle_was_launched,66
1,2,2018-02-09,59.31,cpc,twitter,20180208_us-ca-gb-au_mf_bindle-launch,,bindle_was_launched,51
2,3,2018-02-10,64.79,cpc,twitter,20180208_us-ca-gb-au_mf_bindle-launch,,bindle_was_launched,80


In [47]:
sql = ''' 
SELECT *
FROM users
limit 1;
'''
select(sql)

,id,email,first_name,last_name,country,created_at,age,referrer_id,signup_date,utm_source,utm_campaign,utm_medium,utm_term,utm_content,visitor_id,status,adjust_tracker,adjust_campaign,adjust_adgroup,adjust_creative
0,2305,deborahmueller@collierlesch.net,Arvilla,Wolf,bs,2018-04-12 05:07:09.967302,22,None,2018-04-12,,,,,,76b2205db561a3d9,free,,,,


In [55]:
sql = ''' 
select 
    utm_campaign, 
    count(*), 
    count(
        case
            when status = 'customer' then 1 else null
        end
    ) as customers, 
    100.0 * count(
        case
            when status = 'customer' then 1 else null
        end)  / count(*)
    
from users
where utm_campaign is not null
group by 1
order by 4 desc


'''
select(sql)

,utm_campaign,count,customers,?column?
0,20180201_us_mf_reading,78,24,30.769231
1,20180208_us-ca-gb-au_mf_bindle-launch,300,81,27.000000
2,20180205_ca_mf_online_reading,121,29,23.966942
3,,5583,640,11.463371
